# Restricted versus unrestricted

## High spin states

### Single-reference methods

By a high-spin state, we refer to an open-shell system where all unpaired electrons are chosen to have $\alpha$-spin. As detailed in  section {ref}`sec:spin`, this case is particularly simple since the system can be described by a single Slater determinant.

There are two main ways to proceed:

* Keep the molecular orbitals of the $\alpha$ and $\beta$ spin-orbitals identical but populating the $\alpha$ spin-orbitals with more electrons. This is known as a restricted open-shell calculation.

* Allow for different molecular orbitals for the $\alpha$ and $\beta$ spin-orbitals. This is known as an unrestricted calculation.

Let us illustrate these two approaches on the most important open-shell molecule on earth namely the oxygen molecule.

In [ ]:
import multipsi as mtp
import veloxchem as vlx
au2kcal = 627.51

In [ ]:
mol_str = """
O   0.0  0.0  -0.6
O   0.0  0.0   0.6
"""
molecule = vlx.Molecule.read_molecule_string(mol_str, units="angstrom")
basis = vlx.MolecularBasis.read(molecule, "cc-pvdz")

We determine the triplet ground state with the RODFT and UDFT approaches.

In [ ]:
molecule.set_multiplicity(3)

triplet_rodft_drv = vlx.ScfRestrictedOpenDriver()
triplet_rodft_drv.xcfun = "B3LYP"

triplet_rodft_results = triplet_rodft_drv.compute(molecule, basis)

triplet_udft_drv = vlx.ScfUnrestrictedDriver()
triplet_udft_drv.xcfun = "B3LYP"

triplet_udft_results = triplet_udft_drv.compute(molecule, basis)

Owing to the larger flexibility in the reference state, the UDFT energy is slightly lower than the RODFT one.

In [ ]:
print(f"ROB3LYP energy: {triplet_rodft_drv.get_scf_energy() : 14.8f}")
print(f" UB3LYP energy: {triplet_udft_drv.get_scf_energy() : 14.8f}")

In the RODFT case, we have one set of MOs with the first seven being doubly occupied and MOs eight and nine being singly occupied.

In [ ]:
triplet_rodft_drv.molecular_orbitals.print_orbitals(molecule, basis)

In the UDFT case, there are two different sets of MOs with nine occupied $\alpha$-spin orbitals and seven occupied $\beta$-spin orbitals.

In [ ]:
triplet_udft_drv.molecular_orbitals.print_orbitals(molecule, basis)

It is a common procedure to determine a Kohn–Sham DFT value estimating the expectation value of the  [total spin operator](https://kthpanor.github.io/echem/docs/elec_struct/spin.html#many-electron-systems) in the same manner as would be the case for a Hartree–Fock wave function.

```{note}
Since the [two-particle density](https://kthpanor.github.io/echem/docs/elec_struct/reduced_density.html#two-particle-density) is not directly available in DFT, the expectation value of a two-electron operator such as e.g. the total spin operator is also not available.
```

In [ ]:
print(
    f"<S^2> RODFT = {triplet_rodft_drv.compute_s2(molecule, triplet_rodft_results) : 8.6f}"
)
print(
    f"<S^2> UDFT  = {triplet_udft_drv.compute_s2(molecule, triplet_udft_results) : 8.6f}"
)

The ground state of oxygen is a triplet with spin quantum number $S = 1$ and

$$
\langle \hat{S}^2 \rangle = S(S+1)\hbar^2 = 2 \hbar^2
$$ 
 
This is exactly what we get with a restricted open-shell description, but not in the unrestricted case. In general, an unrestricted state is not a eigenfunction of $\hat{S}^2$ and one refers to this situation by saying that the state is spin contaminated.

The relative simplicity of the unrestricted scheme together with its rather good performance makes it more widely used than the restricted open-shell form.

```{note}
The additional flexibility of unrestricted molecular orbitals does not lower the energy of *closed-shell* restricted states.
```

### Multi-reference methods

A multi-configurational method such as CASSCF can also be used and we note that a CASSCF wave function with only the high-spin open-shell MOs in the active space is equivalent to restricted open-shell Hartree–Fock (ROHF).

In [ ]:
triplet_rohf_drv = vlx.ScfRestrictedOpenDriver()
triplet_rohf_results = triplet_rohf_drv.compute(molecule, basis)

# By default, OrbSpace includes all open-shells in the active space
space = mtp.OrbSpace(molecule, triplet_rohf_drv.mol_orbs)
mcscf_drv = mtp.McscfDriver()
triplet_mc_results = mcscf_drv.compute(molecule, basis, space)

In [ ]:
print(f"       ROHF energy: {triplet_rohf_drv.get_scf_energy() : 14.8f}")
print(f"CASSCF(2,2) energy: {triplet_mc_results["energies"][0] : 14.8f}")

## Low spin states

### Single-reference methods: broken symmetry

To address low spin open-shell systems, we can study [singlet states of the oxygen molecule](https://en.wikipedia.org/wiki/Singlet_oxygen). The two lowest singlet states are $a^1\Delta_g$ and $b^1\Sigma^+_g$ and experimentally they are found to be 22.5 and 37.5 kcal/mol above the ground triplet state, $X^3\Sigma_g^-$, respectively. 

Let us first compute the lowest closed-shell singlet state using DFT.

In [ ]:
molecule.set_multiplicity(1)

singlet_rdft_drv = vlx.ScfRestrictedDriver()
singlet_rdft_drv.xcfun = "B3LYP"
singlet_rdft_results = singlet_rdft_drv.compute(molecule, basis)

The $\pi$-orbitals in the closed-shell state are no longer doubly degenerate.

In [ ]:
singlet_rdft_drv.molecular_orbitals.print_orbitals(molecule, basis)

In [ ]:
print("Closed-shell singlet relative to triplet ground state (in kcal/mol)")
print(
    f"    restricted triplet state: {(singlet_rdft_drv.get_scf_energy() - triplet_rodft_drv.get_scf_energy()) * au2kcal : 14.8f}"
)
print(
    f"  unrestricted triplet state: {(singlet_rdft_drv.get_scf_energy() - triplet_udft_drv.get_scf_energy()) * au2kcal : 14.8f}"
)

These energies match reasonably well with the higher $b^1\Sigma^+_g$ singlet state.

There are two different ways to reach the lower singlet using DFT. The first one is to compute the (negative) excitation energy using TD-DFT.

In [ ]:
rsp_drv = vlx.TdaEigenSolver()
rsp_drv.update_settings({"nstates": 1}, {"xcfun": "B3LYP"})
rsp_results = rsp_drv.compute(molecule, basis, singlet_rdft_results)

In [ ]:
exc_energy = rsp_results["eigenvalues"][0]
print(f"Excitation energy (in kcal/mol): {exc_energy * au2kcal : 14.8}\n")

print("Open-shell singlet energy relative to triplet ground state (in kcal/mol)")
print(
    f"    restricted triplet state: {(singlet_rdft_drv.get_scf_energy() + exc_energy - triplet_rodft_drv.get_scf_energy()) * au2kcal : 14.8f}"
)
print(
    f"  unrestricted triplet state: {(singlet_rdft_drv.get_scf_energy() + exc_energy - triplet_udft_drv.get_scf_energy()) * au2kcal : 14.8f}"
)

These energies is in better agreement with the lowest singlet state. We have here reached the open-shell singlet state by means of a TD-DFT calculation based on a closed-shell reference state that is higher in energy.

Let us instead compute the lower singlet state in a direct manner using unrestricted DFT.

In [ ]:
singlet_udft_drv = vlx.ScfUnrestrictedDriver()
singlet_udft_drv.xcfun = "B3LYP"
singlet_udft_result = singlet_udft_drv.compute(molecule, basis)

In [ ]:
print(f"RDFT singlet: {singlet_rdft_drv.get_scf_energy() : 14.8f}")
print(f"UDFT singlet: {singlet_udft_drv.get_scf_energy() : 14.8f}")

Since the closed-shell state represent a stable point, the UDFT solution will become identical. 

In the absence of magnetic fields, there is no impetus for the MOs of the $\alpha$ and $\beta$ spin-orbitals to differ and the SCF optimization gets stuck in a symmetric solution. In the high-spin case, on the other hand, this symmetry was broken by the fact that we had an excess of $\alpha$-spin electrons.

However, we can break the symmetry by introducing different starting occupations for the $\alpha$ and $\beta$ spin-orbitals and remain in this region during the SCF optimization with the maximum-overlap method.

In [ ]:
# A list of which orbitals are occupied in alpha and beta (starting from the triplet orbitals)
a_occ = [0, 1, 2, 3, 4, 5, 6, 7]
b_occ = [0, 1, 2, 3, 4, 5, 6, 8]

broken_udft_drv = vlx.ScfUnrestrictedDriver()
broken_udft_drv.xcfun = "B3LYP"

broken_udft_drv.maximum_overlap(
    molecule, basis, triplet_rodft_drv.mol_orbs, a_occ, b_occ
)
broken_udft_results = broken_udft_drv.compute(molecule, basis)

We can look at the [natural orbitals](../elec_struct/natural_orbitals) and see that the state indeed corresponds to two singly occupied $\pi$-orbitals.

In [ ]:
natural_orbs = broken_udft_drv.natural_orbitals(broken_udft_results)
natural_orbs.print_orbitals(molecule, basis)

The energy of this singlet relative to the triplet ground state becomes:

In [ ]:
print(
    f"Broken symmetry singlet-triplet gap: {(broken_udft_drv.get_scf_energy() - triplet_udft_drv.get_scf_energy()) * au2kcal : 10.8}"
)

With this trick, we have thus found a lower energy singlet, albeit by introducing spin contamination.

In [ ]:
print(
    f"<S^2> UDFT  = {broken_udft_drv.compute_s2(molecule, broken_udft_results) : 8.6f}"
)

We also note that the energy gap is now well below that of the experimental value. The reason is that, while high spin open-shells are single-determinant in nature, open-shell singlets and low-spin triplets are not. 

A minimum of two determinant is needed for physically correct wave functions

\begin{align*}
| \Psi_\mathrm{S} \rangle & = 
\frac{1}{\sqrt{2}} ( | \alpha \beta \rangle -| \beta \alpha \rangle ) \\
%
| \Psi_\mathrm{T} \rangle & = 
\frac{1}{\sqrt{2}} ( | \alpha \beta \rangle +| \beta \alpha \rangle )
\end{align*}

Our single determinant reference state is seen to correspond to a 50/50 mix of the singlet and triplet wave functions as reflected in the spin contamination of the UDFT state.

True singlets and triplets have spin expectation values of $0$ and $2 \hbar^2$, respectively, whereas we obtained a value close to $\hbar^2$.

However, knowing this fact, we can actually correct the energy. Since we know the energy of the triplet and of the broken-symmetry solution, we can estimate the energy of the true singlet.

In [ ]:
print(
    f"Estimated singlet-triplet gap: {2 * (broken_udft_drv.get_scf_energy() - triplet_udft_drv.get_scf_energy()) * au2kcal : 12.8f}"
)

This is now very close to the experimental energy of 22.5 kcal/mol.

Our technique to obtain the "true" singlet energy is a special case of the more general "weighted-averaged broken symmetry" (WABS) method, which can in principle correct any broken symmetry solution with knowledge of the high-spin energy and the expectation value of the total spin operator.

### Multi-reference solution

Arguably a multi-reference approach is simpler. For O$_2$, we create an active space with two electrons in two orbitals.

In [ ]:
space = mtp.OrbSpace(molecule, singlet_rdft_drv.mol_orbs)
space.cas(2, 2)
singlet_mc_results = mcscf_drv.compute(molecule, basis, space)

In [ ]:
print(
    f"Lowest singlet relative energy: {(singlet_mc_results["energies"][0] - triplet_mc_results["energies"][0]) * au2kcal : 14.8f}"
)

The CASSCF automatically finds the open-shell to be the lowest state.

The CASSCF code in MultiPsi even allows to compute singlets and triplets simultaneously. We only need to deactivate the spin restriction that is applied by defaults to singlets and compute several states at the same time.

In [ ]:
space = mtp.OrbSpace(molecule, singlet_rdft_drv.mol_orbs)
space.spin_restricted = False
space.cas(2, 2)
mcscf_results = mcscf_drv.compute(molecule, basis, space, n_states=4)

In [ ]:
print(
    f"First singlet relative energy : {(mcscf_results["energies"][1] - mcscf_results["energies"][0]) * au2kcal : 14.8f}"
)
print(
    f"First singlet relative energy : {(mcscf_results["energies"][2] - mcscf_results["energies"][0]) * au2kcal : 14.8f}"
)
print(
    f"Second singlet relative energy: {(mcscf_results["energies"][3] - mcscf_results["energies"][0]) * au2kcal : 14.8f}"
)

In a single calculation, the CASSCF finds the lowest triplet state and the two components of the doubly degenerate $a^1\Delta_g$ state followed by a higher lying singlet. 

The energies are not a great match with experiment due to the fact that a CASSCF calculation with so small an active space does not include correlation. A much better result is found by expanding the active space to include all atomic 2p orbitals.

In [ ]:
space.cas(8, 6)
mcscf_results = mcscf_drv.compute(molecule, basis, space, n_states=4)

In [ ]:
print(
    f"First singlet relative energy : {(mcscf_results["energies"][1] - mcscf_results["energies"][0]) * au2kcal : 14.8f}"
)
print(
    f"First singlet relative energy : {(mcscf_results["energies"][2] - mcscf_results["energies"][0]) * au2kcal : 14.8f}"
)
print(
    f"Second singlet relative energy: {(mcscf_results["energies"][3] - mcscf_results["energies"][0]) * au2kcal : 14.8f}"
)

If we examine the CASSCF wave functions, we clearly see the multi-determinant nature of the states.

In [ ]:
for i in range(4):
    print("Wavefunction for state", i + 1)
    mcscf_results["ci_vectors"].print(i)

The determinant notation lists the occupation of the two orbitals in order, with "0" meaning "empty", "a" and "b" meaning respectively singly occupied with an $\alpha$ or $\beta$ electron, and "2" meaning doubly occupied. Clearly in this case, every single state in the calculation is an equal-weight superposition of two determinants.